In [19]:
!pip install torch
!pip install simpletransformers

#### 1) Importing necessary files

In [20]:
import re
import pandas as pd
import numpy as np
import json
import os

In [21]:
from simpletransformers.question_answering import QuestionAnsweringModel

#### 2) Fine Tuning Q&A model on some dummy data.

In [22]:
# Create dummy data to use for training.
train_data = [
    {
        "context": "This is the first context",
        "qas": [
            {
                "id": "00001",
                "is_impossible": False,
                "question": "Which context is this?",
                "answers": [{"text": "the first", "answer_start": 8}],
            }
        ],
    },
    {
        "context": "Other legislation followed, including the Migratory Bird Conservation Act of 1929, a 1937 treaty prohibiting the hunting of right and gray whales,            and the Bald Eagle Protection Act of 1940. These later laws had a low cost to society—the species were relatively rare—and little opposition was raised",
        "qas": [
            {
                "id": "00002",
                "is_impossible": False,
                "question": "What was the cost to society?",
                "answers": [{"text": "low cost", "answer_start": 225}],
            },
            {
                "id": "00003",
                "is_impossible": False,
                "question": "What was the name of the 1937 treaty?",
                "answers": [{"text": "Bald Eagle Protection Act", "answer_start": 167}],
            },
        ],
    },
]

In [23]:
# Save as a JSON file
os.makedirs("data", exist_ok=True)
with open("data/train.json", "w") as f:
    json.dump(train_data, f)

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel(
    "distilbert",
    "distilbert-base-uncased-distilled-squad", use_cuda=False,
    args={"reprocess_input_data": True, "overwrite_output_dir": True,"epochs": 5},
)

# Train the model
model.train_model("data/train.json")

add example index and unique id: 100%|██████████| 3/3 [00:00<00:00, 27654.75it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1, 2.1835758686065674)

In [24]:
# Testing prediction on some dummy data just to ensure that model is working.
to_predict = [
    {
        "context": "This is the context used for demonstrating predictions.",
        "qas": [{"question": "What is this context?", "id": "0"},
               {"question": "What are the predictions?", "id": "1"}],
    }
]

print(model.predict(to_predict,n_best_size=1)[0])

add example index and unique id: 100%|██████████| 2/2 [00:00<00:00, 10824.01it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['This is the context used for demonstrating predictions.']}, {'id': '1', 'answer': ['This is the context used for demonstrating predictions.']}]


#### getJson() function creates a json object with questions to be asked to the Q&A model. The structure of the json is as follows:<br>
1) First value is context, which is the review text.<br>
2) Second is an array of JSON objects, namely qas. This contains an array of JSON objects which are the questions.<br>
3) Each question is a JSON object with two fields, first field titled "question" contains the question, and second field titled "id" should be a unique integer in string format.<br>
4) This function will return the appropriate JSON object for each review and updated counter for id. The JSON object will be used as an input to the Q&A model for predictions.

In [25]:
def getJson(f, count):
    temp = {
      "context": f,
      "qas": [
          {
            "question": "How is the laptop?",
            "id": str(count),
          },
          {
            "question": "How is the support?",
            "id": str(count + 1)
          },
          {
            "question": "How is the display?",
            "id": str(count + 2),
          },
          {
            "question": "How is the battery?",
            "id": str(count + 3),
          },
          {
            "question": "How is the keyboard?",
            "id": str(count + 4),
          },
          {
            "question": "How is the company?",
            "id": str(count + 5),
          },
          {
            "question": "How is the software?",
            "id": str(count + 6),
          },
          {
            "question": "How is the mouse?",
            "id": str(count + 7),
          },
          {
            "question": "How is the OS?",
            "id": str(count + 8),
          },
          {
            "question": "How are the multimedia devices?",
            "id": str(count + 9),
          },
          {
            "question": "How are the graphics?",
            "id": str(count + 10),
          },
          {
            "question": "How is the hard disc?",
            "id": str(count + 11),
          },
          {
            "question": "How is the CPU?",
            "id": str(count + 12),
          },
          {
            "question": "How is the power supply?",
            "id": str(count + 13),
          },
          {
            "question": "How is the memory?",
            "id": str(count + 14),
          },
          {
            "question": "How is the shipping?",
            "id": str(count + 15),
          },
          {
            "question": "How is the motherboard?",
            "id": str(count + 16),
          },
          {
            "question": "How is the hardware?",
            "id": str(count + 17),
          },
          {
            "question": "How are the ports?",
            "id": str(count + 18),
          },
          {
            "question": "How are the optical drivers?",
            "id": str(count + 19),
          },
          {
            "question": "How are the fan?",
            "id": str(count + 20),
          },
          {
            "question": "How is the warranty?",
            "id": str(count + 21),
          },
      ]
    }
    count += 22
    return temp, count

#### This function will clean the input text.

In [26]:
 def preprocess(text_data):
  raw = text_data
  # low = [i.lower() for i in raw]
  low = text_data
  reviews = list()
  for i in low:
    i = re.sub(r'\n|\t|\\', ' ', i)
    reviews.append(i)
  return reviews

In [27]:
df = pd.read_csv('Laptop_SB1_processed_data.csv')

In [28]:
rev = df['reviews']

In [29]:
len(df['reviews'])

450

In [30]:
final = preprocess(rev)

#### Output from Q&A model is again a JSON object. This function parses the JSON and stores output for each aspect in a list which will be used to populate the resulting dataframe. If answer to a particular question is not present in the text as per the model, an empty string is appended to make sure that the number of rows are consistant.

In [31]:
l0 = list()
l1 = list()
l2 = list()
l3 = list()
l4 = list()
l5 = list()
l6 = list()
l7 = list()
l8 = list()
l9 = list()
l10 = list()
l11 = list()
l12 = list()
l13 = list()
l14 = list()
l15 = list()
l16 = list()
l17 = list()
l18 = list()
l19 = list()
l20 = list()
l21 = list()

def compute(fin):
    cnt = 0
    for f in fin:
        js, cnt2 = getJson(f, cnt)
        cnt = cnt2
        pred = model.predict([js], n_best_size=1)[0]
        # print(pred)
        if pred[0]['answer'] is not ['empty']:
            l0.append(pred[0]['answer'][0])
        else:
            l0.append('')
        if pred[1]['answer'] is not ['empty']:
            l1.append(pred[1]['answer'][0])
        else:
            l1.append('')
        if pred[2]['answer'] is not ['empty']:
            l2.append(pred[2]['answer'][0])
        else:
            l2.append('')
        if pred[3]['answer'] is not ['empty']:
            l3.append(pred[3]['answer'][0])
        else:
            l3.append('')
        if pred[4]['answer'] is not ['empty']:
            l4.append(pred[4]['answer'][0])
        else:
            l4.append('')
        if pred[5]['answer'] is not ['empty']:
            l5.append(pred[5]['answer'][0])
        else:
            l5.append('')
        if pred[6]['answer'] is not ['empty']:
            l6.append(pred[6]['answer'][0])
        else:
            l6.append('')
        if pred[7]['answer'] is not ['empty']:
            l7.append(pred[6]['answer'][0])
        else:
            l7.append('')
        if pred[8]['answer'] is not ['empty']:
            l8.append(pred[8]['answer'][0])
        else:
            l8.append('')
        if pred[9]['answer'] is not ['empty']:
            l9.append(pred[9]['answer'][0])
        else:
            l9.append('')
        if pred[10]['answer'] is not ['empty']:
            l10.append(pred[10]['answer'][0])
        else:
            l10.append('')
        if pred[11]['answer'] is not ['empty']:
            l11.append(pred[11]['answer'][0])
        else:
            l11.append('')
        if pred[12]['answer'] is not ['empty']:
            l12.append(pred[12]['answer'][0])
        else:
            l12.append('')
        if pred[13]['answer'] is not ['empty']:
            l13.append(pred[13]['answer'][0])
        else:
            l13.append('')
        if pred[14]['answer'] is not ['empty']:
            l14.append(pred[14]['answer'][0])
        else:
            l14.append('')
        if pred[15]['answer'] is not ['empty']:
            l15.append(pred[15]['answer'][0])
        else:
            l15.append('')
        if pred[16]['answer'] is not ['empty']:
            l16.append(pred[16]['answer'][0])
        else:
            l16.append('')
        if pred[17]['answer'] is not ['empty']:
            l17.append(pred[17]['answer'][0])
        else:
            l17.append('')
        if pred[18]['answer'] is not ['empty']:
            l18.append(pred[18]['answer'][0])
        else:
            l18.append('')
        if pred[19]['answer'] is not ['empty']:
            l19.append(pred[19]['answer'][0])
        else:
            l19.append('')
        if pred[20]['answer'] is not ['empty']:
            l20.append(pred[20]['answer'][0])
        else:
            l20.append('')
        if pred[21]['answer'] is not ['empty']:
            l21.append(pred[21]['answer'][0])
        else:
            l21.append('')

In [32]:
compute(final)

add example index and unique id: 100%|██████████| 22/22 [00:00<00:00, 133731.43it/s]


Running Prediction:   0%|          | 0/3 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 22/22 [00:00<00:00, 150284.51it/s]


Running Prediction:   0%|          | 0/3 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 22/22 [00:00<00:00, 158547.57it/s]


...

Running Prediction:   0%|          | 0/3 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 22/22 [00:00<00:00, 133925.53it/s]


Running Prediction:   0%|          | 0/3 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 22/22 [00:00<00:00, 165961.67it/s]


Running Prediction:   0%|          | 0/3 [00:00<?, ?it/s]

#### Checking if all the lists have the same length. They do have the same length.

In [33]:
print(len(final), len(l0), len(l1), len(l2), len(l3), len(l4), len(l5), len(l6), len(l7), len(l8), len(l9), len(l10), len(l11), len(l12), len(l13), len(l14), len(l15), len(l16), len(l17), len(l18), len(l19), len(l20), len(l21))
# print(len(final), len(l0), len(l1), len(l2), len(l3), len(l4), len(l5), len(l6), len(l7), len(l8))

450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450 450


#### Resulting dataframe.

In [34]:
df2 = pd.DataFrame()

# df2['author_id'] = df['author_id']
df2['reviews'] = final
df2['LAPTOP'] = l0
df2['SUPPORT'] = l1
df2['DISPLAY'] = l2
df2['BATTERY'] = l3
df2['KEYBOARD'] = l4
df2['COMPANY'] = l5
df2['SOFTWARE'] = l6
df2['MOUSE'] = l7
df2['OS'] = l8
df2['MULTIMEDIA_DEVICES'] = l9
df2['GRAPHICS'] = l10
df2['HARD_DISC'] = l11
df2['CPU'] = l12
df2['POWER_SUPPLY'] = l13
df2['MEMORY'] = l14
df2['SHIPPING'] = l15
df2['MOTHERBOARD'] = l16
df2['HARDWARE'] = l17
df2['PORTS'] = l18
df2['OPTICAL_DRIVERS'] = l19
df2['FANS_COOLING'] = l20
df2['WARRANTY'] = l21

In [35]:
df2.head()

,reviews,LAPTOP,SUPPORT,DISPLAY,BATTERY,KEYBOARD,COMPANY,SOFTWARE,MOUSE,OS,...,CPU,POWER_SUPPLY,MEMORY,SHIPPING,MOTHERBOARD,HARDWARE,PORTS,OPTICAL_DRIVERS,FANS_COOLING,WARRANTY
0,Being a PC user my whole life.... This comput...,empty,plenty of storage with 250 gb,graphics card,10 plus hours,computer is absolutely AMAZING!!! 10 plus hour...,empty,super fast processor and really nice graphics ...,super fast processor and really nice graphics ...,empty,...,empty,250 gb,250 gb,computer is absolutely AMAZING!!! 10 plus hour...,empty,super fast processor,empty,super fast processor and really nice graphics ...,empty,empty
1,the laptop was really good and it goes really...,the laptop was really good and it goes really ...,cheap,the laptop,cheap,the laptop was really good and it goes really ...,cheap,cheap,cheap,cheap,...,cheap,cheap,cheap,cheap,laptop,the laptop was really good and it goes really ...,cheap,cheap,any person out there to get this laptop cause ...,cheap
2,"As a lifelong Windows user, I was extremely p...","As a computer science student in college, I fi...","As a computer science student in college, I fi...",empty,"As a computer science student in college, I fi...",empty,computer science student in college,"computer science student in college, I find th...","computer science student in college, I find th...","As a computer science student in college,",...,"As a computer science student in college,",empty,"As a computer science student in college, I fi...","portability, longevity, and ease of use of thi...",backpack,without a big ol' clunky machine in my backpack,"portability, longevity, and ease of use of thi...","As a computer science student in college, I fi...",computer science student in college,"As a computer science student in college, I fi..."
3,Oh my goodness-I am not a happy camper. My HP...,I can't sit,laptop,laptop,scary,My HP is very heavy.,my goodness-I am not a happy camper.,laptop,laptop,My HP is very heavy.,...,My HP is very heavy.,My HP is very heavy.,My HP is very heavy.,My HP is very heavy.,empty,laptop,empty,empty,my goodness-I am not a happy camper.,it was out of warranty
4,"Since I purchased my Toshiba netbook, I have ...","Toshiba netbook,",empty,"The screen takes some getting use to, because ...","Toshiba netbook,","Toshiba netbook,","my Toshiba netbook,","my Toshiba netbook,","my Toshiba netbook,","Toshiba netbook,",...,"Toshiba netbook,","Toshiba netbook,",empty,"my Toshiba netbook,","Toshiba netbook,",a laptob and a desktop,I have a laptob and a desktop.,I have a laptob and a desktop.,"Toshiba netbook,",empty


In [36]:
df2.to_csv('laptop_test.csv', index=False)